# Preprocessing - Combine Database Files

# Summary

This notebook processes raw information and provides at the end a `single CSV file`. This file contains 5 month of vessel-sensor-data, July 2021 until end of October 2021, including a timestamp in minute intervals and 110 other features.

The original raw data was provided within single db-files (.s3db format), containing db-tables where two have been used in this notebook:
* DataLogEntry: containing sensor data;
* DataLog: containing high level information on the different features.

For some features, the sensor data is available in 5 seconds and 10 seconds frequency. However for the majority the data is only available in 1 minute steps. In addition, for some timestamps multiple measurements are logged. In order to reduce sensor data to one measurement per minute, multiple measurements per minute (e.g. 5sec, 10sec, or multiple measurements per minute) the first value was taken.

The dataset further contained column/feature, called EntryState. For EntryState = 0 all values were observed to be 0. It is assumed that for those Values are not available and were set to NaN.

## Instructions
  
1) Copy original Files into repos data folder.

2) If required, adjust folder/dbfiles information in `File setting` section below

3) Run Main Code, CSVs will be saved to repos data folder
    * STAGE_Bluetracker.csv: Combined db-files to one csv;
    * FINAL_Bluetracker.csv: Adjusted Headings and Columns with only NaN values removed > 110 features remaining

---

## Libraries

There are only few Libraries required for this notebook. 
* sqlite3 for reading the db-files using SQL;
* os to read filenames in folder;
* `magic happens all in pandas`.

In [24]:
import sqlite3
import os
import pandas as pd

---

## Basic Functions

Those tree functions are used for:
* Combining the raw db-files (beginning of this notebook);
* Header matching (see end of this notebook).

In [25]:
# Get List with File Names (Full Path) / Loop through given dictionary
def _fGetFullPathFileList(directory_in_str, filetype=".s3db"):
    '''
    This function returns a list. The list includes the full path of files in a given directory that match >filetype< 
    '''
    directory = os.fsencode(directory_in_str)
    files = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(filetype): # or filename.endswith(".py"): 
            #print(f'{directory_in_str}/{filename}')
            files.append(f'{directory_in_str}/{filename}')
            #print(directory)
            #print(os.path.join(directory, filename))
            continue
        else:
            continue
    return files
#print(_fGetFullPathFileList(FILEDIR, '.s3db')[0:5])

# Retruns Db-Table as Dataframe
def _fGetDBTableDataframe(full_dbpath_as_str, tablename_as_str):
    '''
    Returns a Dataframe with all Data from a given database and tablename. Requires 'import sqlite3' library
    '''
    try:
        cnx = sqlite3.connect(full_dbpath_as_str)
        cur = cnx.cursor()
        #print("Database created and Successfully Connected to SQLite")

        # Read table to dataframe
        path = f'SELECT * FROM {tablename_as_str}'
        df = pd.read_sql_query(path, cnx)

        cur.close()

    except sqlite3.Error as error:
        print(f"{full_dbpath_as_str} >> Error while connecting to sqlite:", error)
    finally:
        if cnx:
            cnx.close()

            #print("The SQLite connection is closed")
            return df.copy()

# Combines DB-Tables to one DataFrame. It takes a list of DBfilenames and the tablename as input
def _fCombinedTableDataframe(list_db_paths, table_name='Log_10s', status=False):
    '''
    Write single db tables into one dataframe. Calls _fGetDBTableDataframe
    '''
    print('-'*50)
    print(f'Combining Tables: {table_name}')
    i = 0
    frames = []
    for s3db in list_db_paths:
        df = _fGetDBTableDataframe(s3db, table_name)
        frames = pd.concat([pd.DataFrame(frames), df])
        del df
        # Just some stupid status bar :P
        if status:
            i += 1
            l = len(list_db_paths)
            if i % 6 == 0 or i == l:
                print('|', '#' * i, ' ' * (l - i), '|', frames.shape)

    print(f'Finished combining tables "{table_name}" | Shape: {frames.shape} ')
    return frames

---

## `File Settings`

First we define the Folder path where all our .s3db Files are. Further we define the Table name where the measurements are stored.

In [26]:
# Where are the db-files stored?
FILEDIR = '../data/original'

# Whats the type of the DB file?
DBTYPE = '.s3db'

# What is the relevant Tablename?
TABLE_NAME = 'DataLogEntry'

# Where would you like to save CSV files?
CSVFOLDER = '../data'


---

## `Get Raw data`

Now we can run the code to combine all our single Tables into one dataframe/csv.

In [27]:
if False:
    print(FILEDIR)
    # Run the code to combine all DB Tables / Runtime for 6 month Bluetracker Data = 30ish minutes
    df = _fCombinedTableDataframe(_fGetFullPathFileList(FILEDIR, DBTYPE), TABLE_NAME, True)

    # Save the data to predefined folder
    df.to_csv(f'{CSVFOLDER}/STAGE_Bluetracker.csv', index=False)

Alternative, if you already run the combine code above you can simply load the csv-file. This is way faster.

In [28]:
df = pd.read_csv(f'{CSVFOLDER}/STAGE_Bluetracker.csv')    # 30-40 seconds runtime
print(f'Dataframe shape: {df.shape}')

Dataframe shape: (39440874, 5)


In [29]:
# Delete EntryState = 0 Values
df = df[df.EntryState==2].copy()
df.drop('EntryState', inplace=True, axis=1)
print(f'Dataframe shape after dropping EntryState==0: {df.shape}')

Dataframe shape after dropping EntryState==0: (38422797, 4)


In [30]:
# Get rid of seconds for Entry Date
df['EntryDate'] = df.EntryDate.apply(lambda row: str(row)[0:16])

# Change EntryDate to datetime type
df['EntryDate'] = pd.to_datetime(df['EntryDate'])

In [31]:
# Create empty Dataframe
df_dates = pd.DataFrame()

# Create Date Values and rename Column
df_dates['EntryDate'] = pd.date_range(start=df['EntryDate'].min(), end=df['EntryDate'].max(), freq='T')


In [32]:
df.drop_duplicates(subset=['EntryDate', 'DataLogID'], keep='first', inplace=True)
print(f'Dataframe shape after dropping duplicates: {df.shape}')

Dataframe shape after dropping duplicates: (23380630, 4)


In [33]:
df.drop(columns='DataLogEntryID', inplace=True)

In [34]:
list_logid = list(df.DataLogID.unique())

In [35]:
# Write DataLogIDs to own column
for LOGID in list_logid:
    df_dates = pd.merge(df_dates, df[df.DataLogID == LOGID].drop(columns='DataLogID'), how='left', on='EntryDate')
    df_dates.rename(columns={"EntryValue": LOGID}, inplace=True)
df = df_dates.copy()
del df_dates
print(f'Dataframe shape after dropping duplicates: {df.shape}')

Dataframe shape after dropping duplicates: (220321, 123)


In [36]:
df.head()

,EntryDate,11014,11011,11015,11020,11017,11021,14003,11012,11016,...,12070,12061,12062,14006,13004,14005,13002,13000,13003,13001
0,2021-05-31 00:00:00,65.0,0.0,-0.004,28.0,0.0,-0.002,1.622416e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-31 00:01:00,58.0,0.0,-0.003,25.0,0.0,-0.002,1.622416e+09,86402.0,1.343,...,192.031204,293.132843,290.477203,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-31 00:02:00,63.0,0.0,0.007,27.0,0.0,0.000,1.622416e+09,86402.0,1.343,...,193.393600,291.190186,289.918213,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-05-31 00:03:00,63.0,0.0,0.008,27.0,0.0,-0.001,1.622416e+09,86402.0,1.343,...,194.051834,290.139618,291.215454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-05-31 00:04:00,64.0,0.0,0.000,27.0,0.0,-0.001,1.622416e+09,86402.0,1.343,...,195.484314,288.614624,285.923615,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

## Replace Column /Feature Names

The information about the Features/Sensors/Columns are stored in the db-files in a Table called `DataLog`.

In [ ]:
# Get some db file path + name
files = _fGetFullPathFileList(FILEDIR, DBTYPE)
print(files[0])

# Read to dataframe
df_heading = _fGetDBTableDataframe(files[0], 'DataLog')

In [ ]:
df_heading.head()

### Duplicate VarNames

Unfortunately, som VarNames appearing multiple times.

In [ ]:
df_heading.VarName.value_counts().head(6)

In [40]:
dic_varname = {13002: 'ME.LOD.act.PRC.trend', \
    13003: 'ME.SFC.act.gPkWh.trend', \
        13004: 'AE.SFC.act.gPkWh.trend', \
            13006: 'V.SLPTW.act.PRC.trend', \
                14005: 'V.STW.act.kn.trend'}

In [ ]:
df_heading[df_heading.VarName=='ME.LOD.act.PRC']

We will match now the DataLogID with the VarName and replace the column/feature names by `VarName`.

In [42]:
# Rename Columns
for LOGID in list_logid:
    if LOGID in dic_varname:
        VARNAME = dic_varname[LOGID]
    else:
        VARNAME = df_heading[df_heading.DataLogID == LOGID].VarName.unique()[0]
    df.rename(columns={LOGID: VARNAME}, inplace=True)

In [ ]:
df.head()

In [44]:
df.shape

(220321, 123)

Now we are ready and can save our DataFrame to our FINAL_Bluetracker.csv file.

In [45]:
df.sort_values('EntryDate', inplace=True)
df.to_csv(f'{CSVFOLDER}/FINAL_Bluetracker_full.csv', index=False)

---

In [23]:
df.isna().sum()

EntryDate                          0
LM1.plc_cpuload.avg_5s.PRC      8642
LM1.plc_skewtime.act.s          8642
LM1.plc_timeBalance.act.s       8642
LM2.plc_cpuload.avg_5s.PRC      8642
                               ...  
V.STW.act.kn.trend            220319
ME.LOD.act.PRC.trend          220319
ME.POW.act.MW                 220319
ME.SFC.act.gPkWh.trend        220319
ME.SPD.act.rpm                220319
Length: 123, dtype: int64